Dan Pelis\
AAI 628
# Monet Style Transfer
Input images are 256x256

In [34]:
import cv2
import os
from tqdm import tqdm
from tensorflow.keras.optimizers import Adam
from keras.layers import BatchNormalization, Reshape, Dense, Input, LeakyReLU, Conv2D, Conv2DTranspose, Concatenate, ReLU, Dropout, ZeroPadding2D
from keras.models import Model
from keras.initializers import RandomNormal
# from keras.optimizers import adam
import numpy as np
import time

In [35]:
def load_img(file_path):
    img = cv2.imread(file_path)
    if np.random.rand() > 0.5:
        img = cv2.flip(img, 1)
    img = (img/127.5) - 1
    return img

In [36]:
images = []
image_path = './images'

for files in tqdm(os.listdir(image_path)):
    file_path = os.path.join(image_path, files)
    input_img = load_img(file_path)
    images.append(input_img)
    
images = np.array(images)

100%|██████████████████████████████████████████████████████████████████████████████████████████| 7038/7038 [00:23<00:00, 295.47it/s]


In [38]:
images.shape

(7038, 256, 256, 3)

Resources\
https://machinelearningmastery.com/how-to-develop-cyclegan-models-from-scratch-with-keras/